In [1]:
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import helpers

train_df = pd.read_csv('data/train.csv')

print(train_df.shape)

pd.set_option('display.max_rows', 500)

missing_counts = train_df.isnull().sum()
print(missing_counts)

# I cross-referenced missing values with expected missing values from the description, electrical has one unexpected
# missing value, most likely a data entry error. Imputing via mode. This indicates that the dataset is susceptible
# to human error, to be accounted for in data pipeline.

mode_electrical = train_df['Electrical'].mode()
train_df['Electrical'] = train_df['Electrical'].fillna(mode_electrical)

total_cells = np.prod(train_df.shape)
total_missing = missing_counts.sum()
print(f"Total missing value count: {total_missing}")
print(f"Percentage of total values that are missing: {100 * total_missing / total_cells:.2f}%")

# LotFrontal has unexpected NA values.
# First, I check to see if there's any explainable correlations as to why (townhouse, apartment etc..?)
# I know that to get the most meaningful correlations, categories must be encoded first.

correlation = train_df["LotFrontage"].corr(train_df["SalePrice"])
print(f"Correlation of LotFrontage vs SalePrice: {correlation}")

# 0.35, a significant correlation.

# Checking for minimum LotFrontage in case there's 0's, would give insight into NA LotFrontage.
print(min(train_df["LotFrontage"]))

train_df = helpers.init_fill_na(train_df)
# Fills in all na values except for LotFrontage, kept this here for reproducibility of my EDA process.

(1460, 81)
Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType        872
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0

In [2]:
# As I'm adjusting this, LotArea as expected has had a high correlation. I expect squaring it to yield an even
# higher correlation value to which I was correct.
train_df['Missing_LotFrontage'] = train_df['LotFrontage'].isna().astype(int)
train_df['SqrtLotArea'] = np.sqrt(train_df['LotArea'])
# Should pass off as remainder, easy to compare expected correlation results.
train_df['Sqrt1stFlrSF'] = np.sqrt(train_df['1stFlrSF'])

# No columns to be dropped, dataframe to be kept dense for correlation analysis.
model_pipeline = Pipeline(steps=[
    ('preprocessor', helpers.generate_preprocessor(
        drop=None,
        sparse_output=False,
        ordinal_cats_ordered=helpers.get_ordinal_cats_ordered(),
        categorical_cols_ordinal=helpers.get_categorical_cols_ordinal(),
        numerical_cols=helpers.get_numeric_cols(),
        categorical_cols_nominal=helpers.get_categorical_cols_nominal()
    ))
])

model_pipeline.set_output(transform="pandas")

# Model fitting
X_train_transformed = model_pipeline.fit_transform(train_df)

correlation_matrix = X_train_transformed.corr()

def get_correlations(col_name):
    if col_name in correlation_matrix:
        print(f"Extracting correlations with '{col_name}'...")
        correlations = correlation_matrix[col_name].copy()
        sorted_lf_corr = correlations.sort_values(ascending=False)
    
        print(f"\nMost positively correlated features with {col_name}:")
        print(sorted_lf_corr.drop(col_name).head(20)) # Show top 20 positive
    
        print(f"\nMost negatively correlated features with {col_name}:")
        print(sorted_lf_corr.tail(20)) # Show top 20 negative
    else:
        print(f"Column '{col_name}' not found in correlation matrix.")

get_correlations('remainder__Missing_LotFrontage')
get_correlations('num__LotFrontage')


Extracting correlations with 'remainder__Missing_LotFrontage'...

Most positively correlated features with remainder__Missing_LotFrontage:
ord__LotShape                0.308859
ohe__LotConfig_CulDSac       0.236144
ohe__LandContour_Low         0.145837
remainder__SqrtLotArea       0.141870
ohe__Exterior2nd_Plywood     0.138022
num__Fireplaces              0.136967
ohe__Neighborhood_ClearCr    0.131162
ohe__Neighborhood_Gilbert    0.126692
ohe__Neighborhood_NWAmes     0.123814
ohe__Exterior1st_Plywood     0.122223
num__LotArea                 0.121964
ohe__Exterior2nd_HdBoard     0.119655
ohe__SaleCondition_Normal    0.114375
ohe__Exterior1st_HdBoard     0.112939
ord__LandSlope               0.109453
ohe__MSZoning_RL             0.108929
ohe__SaleType_WD             0.107133
ohe__MasVnrType_BrkFace      0.105394
ohe__Neighborhood_Sawyer     0.105221
ohe__Condition1_PosN         0.104882
Name: remainder__Missing_LotFrontage, dtype: float64

Most negatively correlated features with remain

In [3]:
# LotShape, LotConfig were the most correlated with missing LotFrontage values, but doesn't explain the missing values.
# Attempting to break down LotShape further by temporarily making it OHE to see if a component of it yields a stronger
# correlation in an attempt to find an explanation.

ordinal_cats_ordered = helpers.get_ordinal_cats_ordered()
categorical_cols_ordinal = helpers.get_categorical_cols_ordinal()
del ordinal_cats_ordered[0]
del categorical_cols_ordinal[0]

model_pipeline = Pipeline(steps=[
    ('preprocessor', helpers.generate_preprocessor(
        drop=None,
        sparse_output=False,
        ordinal_cats_ordered=ordinal_cats_ordered,
        categorical_cols_ordinal=categorical_cols_ordinal,
        numerical_cols=helpers.get_numeric_cols() + ["SalePrice"], #used for analysis
        categorical_cols_nominal=helpers.get_categorical_cols_nomial() + ["LotShape"]
    ))
])
model_pipeline.set_output(transform="pandas")

X_train_transformed = model_pipeline.fit_transform(train_df)
correlation_matrix = X_train_transformed.corr()

get_correlations('remainder__Missing_LotFrontage')
get_correlations('num__LotFrontage')

# -0.335 correlation for LotShape_Reg for missing values.
# This confirms, LotFrontage is unexplained, given the correlations, strategy will be to impute via
# median of "LotFrontage/sqrt(LotArea)" GROUPED BY (possible bins of Sqrt1stFlrSF), BldgType, LotShape, and MSZoning.


Extracting correlations with 'remainder__Missing_LotFrontage'...

Most positively correlated features with remainder__Missing_LotFrontage:
ohe__LotShape_IR1            0.309042
ohe__LotConfig_CulDSac       0.236144
ohe__LandContour_Low         0.145837
remainder__SqrtLotArea       0.141870
ohe__Exterior2nd_Plywood     0.138022
num__Fireplaces              0.136967
ohe__Neighborhood_ClearCr    0.131162
ohe__Neighborhood_Gilbert    0.126692
ohe__Neighborhood_NWAmes     0.123814
ohe__Exterior1st_Plywood     0.122223
num__LotArea                 0.121964
ohe__Exterior2nd_HdBoard     0.119655
ohe__SaleCondition_Normal    0.114375
ohe__Exterior1st_HdBoard     0.112939
ord__LandSlope               0.109453
ohe__MSZoning_RL             0.108929
ohe__SaleType_WD             0.107133
ohe__MasVnrType_BrkFace      0.105394
ohe__Neighborhood_Sawyer     0.105221
ohe__Condition1_PosN         0.104882
Name: remainder__Missing_LotFrontage, dtype: float64

Most negatively correlated features with remain

In [4]:
# Getting output names for BldgType and MSZoning, to see what percentage of the total distribution per category
# is NA with respect to LotFrontage.

transformers = model_pipeline.named_steps['preprocessor'].transformers_
bldg_type_list = []
mszone_list = []

for name, fitted_transformer, input_cols in transformers:
    if name == "ohe":
        out_features = fitted_transformer.get_feature_names_out()
        bldg_type_list = ["ohe__" + item for item in out_features if item.startswith("BldgType")]
        mszone_list = ["ohe__" + item for item in out_features if item.startswith("MSZoning")]
        break

print(bldg_type_list)
print(mszone_list)

filtlered_df = X_train_transformed[X_train_transformed['remainder__Missing_LotFrontage'] == 1]
for item in bldg_type_list:
    total_missing = sum(filtlered_df[item])
    total = sum(X_train_transformed[item])
    print(f"Total proportion for {item} is: {total_missing/total * 100:.2f}")
for item in mszone_list:
    total_missing = sum(filtlered_df[item])
    total = sum(X_train_transformed[item])
    print(f"Total proportion for {item} is: {total_missing/total * 100:.2f}")

# Results suggest we can get strong medians by grouping for imputation for LotFrontage/sqrt(LotArea) by BldgType & MsZone!

['ohe__BldgType_1Fam', 'ohe__BldgType_2fmCon', 'ohe__BldgType_Duplex', 'ohe__BldgType_Twnhs', 'ohe__BldgType_TwnhsE']
['ohe__MSZoning_C (all)', 'ohe__MSZoning_FV', 'ohe__MSZoning_RH', 'ohe__MSZoning_RL', 'ohe__MSZoning_RM']
Total proportion for ohe__BldgType_1Fam is: 18.52
Total proportion for ohe__BldgType_2fmCon is: 9.68
Total proportion for ohe__BldgType_Duplex is: 9.62
Total proportion for ohe__BldgType_Twnhs is: 6.98
Total proportion for ohe__BldgType_TwnhsE is: 19.30
Total proportion for ohe__MSZoning_C (all) is: 0.00
Total proportion for ohe__MSZoning_FV is: 12.31
Total proportion for ohe__MSZoning_RH is: 18.75
Total proportion for ohe__MSZoning_RL is: 19.90
Total proportion for ohe__MSZoning_RM is: 8.72


In [5]:
train_df = pd.read_csv('data/train.csv')
train_df = helpers.init_fill_na(train_df)
print(type(train_df))
df = helpers.fill_na_lotfrontage(train_df)
filtered_df = df[]


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
      ohe__BldgType_1Fam  ohe__BldgType_2fmCon  ohe__BldgType_Duplex  \
7                    1.0                   0.0                   0.0   
12                   1.0                   0.0                   0.0   
14                   1.0                   0.0                   0.0   
16                   1.0                   0.0                   0.0   
24                   1.0                   0.0                   0.0   
31                   1.0                   0.0                   0.0   
42                   1.0                   0.0                   0.0   
43                   1.0                   0.0                   0.0   
50                   1.0                   0.0                   0.0   
64                   1.0                   0.0                   0.0   
66                   1.0                   0.0                   0.0   
76                   1.0                   0.0              